In [1]:
from dotenv import load_dotenv
import os

# loads variables from .env
load_dotenv()

True

In [3]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if GOOGLE_API_KEY is None:
    raise ValueError("Environment variable GOOGLE_API_KEY not found!")

print("Secret loaded successfully!")

Secret loaded successfully!


In [5]:
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
print("✅ Gemini API key setup complete.")

✅ Gemini API key setup complete.


In [52]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [8]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504], # Retry on these HTTP errors
)

---
## 🛣️ Section 4: Parallel Workflows - Independent Researchers

**The Problem: The Bottleneck**

The previous sequential agent is great, but it's an assembly line. Each step must wait for the previous one to finish. What if you have several tasks that are **not dependent** on each other? For example, researching three *different* topics. Running them in sequence would be slow and inefficient, creating a bottleneck where each task waits unnecessarily.

**The Solution: Concurrent Execution**

When you have independent tasks, you can run them all at the same time using a `ParallelAgent`. This agent executes all of its sub-agents concurrently, dramatically speeding up the workflow. Once all parallel tasks are complete, you can then pass their combined results to a final 'aggregator' step.

**Use Parallel when:** Tasks are independent, speed matters, and you can execute concurrently.

To learn more, check out the documentation related to [parallel agents in ADK](https://google.github.io/adk-docs/agents/workflow-agents/parallel-agents/).

**Architecture: Multi-Topic Research**

```mermaid
graph TD
    A["User Request: Research 3 topics"] -- > B["Parallel Execution"]
    B -- > C["Tech Researcher"]
    B -- > D["Health Researcher"]
    B -- > E["Finance Researcher"]

    C -- > F["Aggregator"]
    D -- > F
    E -- > F
    F -- > G["Combined Report"]

    style B fill:#ffffcc
    style F fill:#ffccff
```

<img width="600" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/parallel-agent.png" alt="Parallel Agent" />

### 4.1 Example: Parallel Multi-Topic Research

Let's build a system with four agents:

1. **Tech Researcher** - Researches AI/ML news and trends
2. **Health Researcher** - Researches recent medical news and trends
3. **Finance Researcher** - Researches finance and fintech news and trends
4. **Aggregator Agent** - Combines all research findings into a single summary

In [17]:
# Tech Researcher: Focuses on AI and ML trends.
tech_researcher = Agent(
    name="TechResearcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Research the latest AI/ML trends. Include 3 key developments,
the main companies involved, and the potential impact. Keep the report very concise (100 words).""",
    tools=[google_search],
    output_key="tech_research",  # The result of this agent will be stored in the session state with this key.
)

print("✅ tech_researcher created.")

✅ tech_researcher created.


In [19]:
# Health Researcher: Focuses on medical breakthroughs.
health_researcher = Agent(
    name="HealthResearcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Research recent medical breakthroughs. Include 3 significant advances,
their practical applications, and estimated timelines. Keep the report concise (100 words).""",
    tools=[google_search],
    output_key="health_research",  # The result will be stored with this key.
)

print("✅ health_researcher created.")

✅ health_researcher created.


In [21]:
# Finance Researcher: Focuses on fintech trends.
finance_researcher = Agent(
    name="FinanceResearcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Research current fintech trends. Include 3 key trends,
their market implications, and the future outlook. Keep the report concise (100 words).""",
    tools=[google_search],
    output_key="finance_research",  # The result will be stored with this key.
)

print("✅ finance_researcher created.")

✅ finance_researcher created.


In [23]:
# The AggregatorAgent runs *after* the parallel step to synthesize the results.
aggregator_agent = Agent(
    name="AggregatorAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # It uses placeholders to inject the outputs from the parallel agents, which are now in the session state.
    instruction="""Combine these three research findings into a single executive summary:

    **Technology Trends:**
    {tech_research}
    
    **Health Breakthroughs:**
    {health_research}
    
    **Finance Innovations:**
    {finance_research}
    
    Your summary should highlight common themes, surprising connections, and the most important key takeaways from all three reports. The final summary should be around 200 words.""",
    output_key="executive_summary",  # This will be the final output of the entire system.
)

print("✅ aggregator_agent created.")

✅ aggregator_agent created.


👉 **Then we bring the agents together under a parallel agent, which is itself nested inside of a sequential agent.**

This design ensures that the research agents run first in parallel, then once all of their research is complete, the aggregator agent brings together all of the research findings into a single report:

In [26]:
# The ParallelAgent runs all its sub-agents simultaneously.
parallel_research_team = ParallelAgent(
    name="ParallelResearchTeam",
    sub_agents=[tech_researcher, health_researcher, finance_researcher],
)

# This SequentialAgent defines the high-level workflow: run the parallel team first, then run the aggregator.
root_agent = SequentialAgent(
    name="ResearchSystem",
    sub_agents=[parallel_research_team, aggregator_agent],
)

print("✅ Parallel and Sequential Agents created.")

✅ Parallel and Sequential Agents created.


In [28]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Run the daily executive briefing on Tech, Health, and Finance"
)


 ### Created new session: debug_session_id

User > Run the daily executive briefing on Tech, Health, and Finance
TechResearcher > **Key AI/ML Trends for 2025**

**1. Generative AI Expansion:** Generative AI is moving beyond text and image creation to applications like drug discovery, new material invention, and complex protein design. Companies like Google (Gemini) and OpenAI are at the forefront of this evolution, with potential impacts ranging from accelerated drug development to scientific breakthroughs.

**2. Agentic AI and Automation:** AI systems are becoming more autonomous, capable of independent action and decision-making without constant human oversight. This "Agentic AI" is poised to revolutionize fields like healthcare, logistics, and finance by enabling proactive problem-solving.

**3. Multimodal AI and Enhanced Understanding:** AI models are increasingly adept at processing and integrating information from various sources, including text, images, and audio. This multimod

In [48]:
from IPython.display import display, Image, Markdown

text = ""
for _response in response:
    for part in _response.content.parts:
        text += part.text

Markdown(text)

**Key AI/ML Trends for 2025**

**1. Generative AI Expansion:** Generative AI is moving beyond text and image creation to applications like drug discovery, new material invention, and complex protein design. Companies like Google (Gemini) and OpenAI are at the forefront of this evolution, with potential impacts ranging from accelerated drug development to scientific breakthroughs.

**2. Agentic AI and Automation:** AI systems are becoming more autonomous, capable of independent action and decision-making without constant human oversight. This "Agentic AI" is poised to revolutionize fields like healthcare, logistics, and finance by enabling proactive problem-solving.

**3. Multimodal AI and Enhanced Understanding:** AI models are increasingly adept at processing and integrating information from various sources, including text, images, and audio. This multimodal capability leads to more natural human-computer interactions and sophisticated applications across industries.

**Main Companies Involved:** Major players like Google, Microsoft, NVIDIA, OpenAI, Anthropic, and Meta are heavily investing in and developing these AI/ML trends.

**Potential Impact:** These advancements promise significant gains in efficiency, innovation, and personalization across numerous sectors. However, they also raise critical considerations regarding ethics, data privacy, and the need for robust governance frameworks.Here are three significant recent medical breakthroughs:

1.  **AI in Drug Discovery:** Artificial intelligence is revolutionizing drug development by accelerating target identification, optimizing drug candidates, and predicting toxicity. This has already led to AI-designed drugs entering clinical trials with significantly higher success rates than traditional methods. Practical applications include faster development of novel therapeutics and repurposing existing drugs for new diseases. Timelines for widespread impact are already underway, with many AI-developed drugs in various trial phases.

2.  **mRNA Vaccine Technology:** Building on the success of COVID-19 vaccines, mRNA technology is rapidly advancing. Its adaptability and speed allow for quicker development of vaccines against infectious diseases like influenza and RSV, as well as promising applications in cancer immunotherapy and treating genetic disorders. Practical applications include faster pandemic response and personalized cancer treatments. Further development and broader clinical translation are expected within the next 1-5 years.

3.  **CRISPR Gene Editing Advancements:** CRISPR technology continues to evolve, moving beyond basic gene editing. New applications include base editors and RNA editors, offering more precise genetic modifications. Clinical trials for inherited diseases like sickle cell anemia have shown success, and research is expanding into treating cancer and other complex conditions. Practical applications are already emerging for genetic disorders, with broader therapeutic uses anticipated in the next 5-10 years.Here's your daily briefing for November 12, 2025:

**Technology:**
*   China's robotics industry saw a 29.5% revenue surge in the first three quarters of 2025. Microsoft has secured approval to export Nvidia chips to the UAE.
*   AI is increasingly integrated into business operations, with On sportswear company leveraging AI for cash forecasting, unlocking $350 million in previously underutilized cash.
*   Companies like Cisco are scheduled to release their Q1 fiscal year 2026 financial results today.

**Health:**
*   November 12th is World Pneumonia Day, with the WHO highlighting related information.
*   The Illinois Department of Public Health is hosting a summit on the impact of climate change and extreme weather on public health on November 12-13.
*   Aethlon Medical will release its fiscal second quarter financial results and host a conference call today, discussing their medical therapeutic developments.

**Finance:**
*   Several companies, including flyExclusive, NTG Clarity, and Kopin Corporation, are set to announce their third-quarter 2025 financial results today, November 12, 2025. Cisco will also announce its Q1 FY2026 financial results.
*   Oman is positioning itself as a leader in the green steel industry, leveraging its renewable energy and hydrogen projects for sustainable production.
*   The sportswear company On has enhanced its cash management through AI, significantly increasing its investable capital.## Executive Summary: AI-Driven Innovation Accelerates Across Tech, Health, and Finance

Artificial Intelligence is the unifying force driving significant advancements across technology, health, and finance. Generative AI is expanding beyond content creation into scientific discovery, with applications in drug discovery and material science already showing promise. This dovetails with breakthroughs in health, where AI is accelerating drug development, and mRNA technology and CRISPR gene editing are ushering in new eras for vaccines and genetic therapies.

In finance, AI is enhancing operational efficiency, as demonstrated by On sportswear company's AI-driven cash forecasting that unlocked substantial capital. The integration of AI into business operations is a key trend, impacting areas from financial reporting to logistics. The rise of "Agentic AI" and multimodal AI further underscores AI's increasing autonomy and sophisticated understanding, promising proactive problem-solving and more natural human-computer interactions. While these innovations offer immense potential for efficiency and personalization, they necessitate careful consideration of ethical implications and data privacy.

🎉 Great! You've seen how parallel agents can dramatically speed up workflows by running independent tasks concurrently.

So far, all our workflows run from start to finish and then stop. **But what if you need to review and improve an output multiple times?** Next, we'll build a workflow that can loop and refine its own work.